# __Image augmentation__

Image (tiff) augmentation to increase training sets <br>

Methods: <br>
random rotation, flipping, zoom, and shear

In [1]:
import os
import numpy as np
import cv2
from skimage import io, img_as_ubyte

In [2]:
def augment_image(image, rotation_range=2, flip_probability=0.5):
    """
    Augment the input image with random rotation, flipping, zoom, and shear.
    
    Args:
    - image: numpy array of shape (height, width, channels)
    - rotation_range: maximum rotation angle in degrees
    - flip_probability: probability of flipping the image horizontally or vertically
    
    Returns:
    - augmented_image: numpy array of the same shape as input image
    """
    # Get image dimensions
    height, width = image.shape[:2]
    
    # Random rotation
    angle = np.random.uniform(-rotation_range, rotation_range)
    rotation_matrix = cv2.getRotationMatrix2D((width/2, height/2), angle, 1)
    
    # Determine the new bounds
    cos = np.abs(rotation_matrix[0, 0])
    sin = np.abs(rotation_matrix[0, 1])
    new_width = int((height * sin) + (width * cos))
    new_height = int((height * cos) + (width * sin))
    
    # Adjust the rotation matrix
    rotation_matrix[0, 2] += (new_width / 2) - (width / 2)
    rotation_matrix[1, 2] += (new_height / 2) - (height / 2)
    
    # Perform the rotation
    augmented_image = cv2.warpAffine(image, rotation_matrix, (new_width, new_height), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)
    
    # Random horizontal flip
    if np.random.random() < flip_probability:
        augmented_image = cv2.flip(augmented_image, 1)
    
    # Random vertical flip
    if np.random.random() < flip_probability:
        augmented_image = cv2.flip(augmented_image, 0)
    
    # Random zoom
    if np.random.random() < 0.5:
        zoom_factor = np.random.uniform(0.8, 1.2)
        
        if zoom_factor > 1:  # Zoom in
            # Calculate crop dimensions
            crop_height = int(height / zoom_factor)
            crop_width = int(width / zoom_factor)
            
            # Calculate crop starting points
            start_y = (height - crop_height) // 2
            start_x = (width - crop_width) // 2
            
            # Ensure valid crop coordinates
            start_y = max(0, start_y)
            start_x = max(0, start_x)
            end_y = min(height, start_y + crop_height)
            end_x = min(width, start_x + crop_width)
            
            # Crop and resize
            cropped = augmented_image[start_y:end_y, start_x:end_x]
            augmented_image = cv2.resize(cropped, (width, height), interpolation=cv2.INTER_LINEAR)
        
        else:  # Zoom out
            # Resize to smaller dimension
            new_height = int(height * zoom_factor)
            new_width = int(width * zoom_factor)
            resized = cv2.resize(augmented_image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
            
            # Create a blank canvas of original size
            result = np.zeros_like(image)
            
            # Calculate padding
            pad_y = (height - new_height) // 2
            pad_x = (width - new_width) // 2
            
            # Place the resized image in the center
            y1 = max(0, pad_y)
            y2 = min(height, pad_y + new_height)
            x1 = max(0, pad_x)
            x2 = min(width, pad_x + new_width)
            
            result[y1:y2, x1:x2] = resized[:y2-y1, :x2-x1]
            augmented_image = result
    
    # Random shear
    if np.random.random() < 0.5:
        shear_factor = np.random.uniform(-0.3, 0.3)
        M = np.float32([[1, shear_factor, 0], [0, 1, 0]])
        augmented_image = cv2.warpAffine(augmented_image, M, (width, height), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)
    
    return augmented_image

In [3]:
def augment_and_save_image(folder_path, image_name, output_folder=None, counter=1):
    """
    Read an image from a folder, augment it, and save it back to the same or specified folder.
    
    Args:
    - folder_path: string, path to the folder containing the image
    - image_name: string, name of the image file
    - output_folder: string, path to the output folder (if None, use the same folder)
    - counter: int, a counter to make the filename unique
    
    Returns:
    - None
    """
    # Construct full path to the image
    image_path = os.path.join(folder_path, image_name)
    
    # Read the image using scikit-image
    original_image = io.imread(image_path)
    
    if original_image is None:
        print(f"Error: Could not read image {image_name}")
        return
    
    # Augment the image
    augmented_image = augment_image(original_image)
    
    if augmented_image is None:
        print("Error: Augmentation failed")
        return
    
    # Convert to 8-bit if necessary
    if augmented_image.dtype != np.uint8:
        augmented_image = img_as_ubyte(augmented_image / augmented_image.max())
    
    # Generate a new filename for the augmented image
    name, ext = os.path.splitext(image_name)
    augmented_name = f"{name}_aug_{counter:03d}{ext}"
    
    # Determine output folder
    if output_folder is None:
        output_folder = folder_path
    
    augmented_path = os.path.join(output_folder, augmented_name)
    
    # Save the augmented image using scikit-image
    try:
        io.imsave(augmented_path, augmented_image)
        print(f"Augmented image saved successfully as {augmented_name}")
    except Exception as e:
        print(f"Error saving image: {str(e)}")

In [4]:
def generate_multiple_augmentations(folder_path, n, output_folder=None):
    """
    Generate multiple augmented images from all images in a folder.
    
    Args:
    - folder_path: string, path to the folder containing the images
    - n: int, total number of images to generate (including original images)
    - output_folder: string, path to the output folder (if None, use the same folder)
    
    Returns:
    - None
    """
    # Get list of image files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff'))]
    
    if not image_files:
        print("No image files found in the specified folder.")
        return
    
    # Create output folder if it doesn't exist
    if output_folder is None:
        output_folder = folder_path
    os.makedirs(output_folder, exist_ok=True)
    
    # Calculate number of augmentations per image
    num_orig_images = len(image_files)
    augmentations_per_image = max(0, (n - num_orig_images) // num_orig_images)
    extra_augmentations = max(0, (n - num_orig_images) % num_orig_images)
    
    print(f"Generating {augmentations_per_image} augmentations per image, plus {extra_augmentations} extra.")
    
    # Generate augmentations
    augmentation_counter = 1
    for i, image_name in enumerate(image_files):
        # Copy original image to output folder
        original_path = os.path.join(folder_path, image_name)
        io.imsave(os.path.join(output_folder, image_name), io.imread(original_path))
        
        # Generate augmentations
        for j in range(augmentations_per_image + (1 if i < extra_augmentations else 0)):
            augment_and_save_image(folder_path, image_name, output_folder, augmentation_counter)
            augmentation_counter += 1
    
    print(f"Augmentation complete. {n} total images generated in {output_folder}")

In [5]:
folder_path = '../data/train/jpg/B/'
output_folder = '../data/train/jpg/B/'  # Optional
generate_multiple_augmentations(folder_path, 100, output_folder)

Generating 1 augmentations per image, plus 4 extra.
Augmented image saved successfully as E02_t0_NaiveCult_s10_channel3_aug_001.jpg
Augmented image saved successfully as E02_t0_NaiveCult_s10_channel3_aug_002.jpg
Augmented image saved successfully as E02_t0_NaiveCult_s19_channel3_aug_003.jpg
Augmented image saved successfully as E02_t0_NaiveCult_s19_channel3_aug_004.jpg
Augmented image saved successfully as E02_t0_NaiveCult_s26_channel3_aug_005.jpg
Augmented image saved successfully as E02_t0_NaiveCult_s26_channel3_aug_006.jpg
Augmented image saved successfully as E02_t0_NaiveCult_s31_channel3_aug_007.jpg
Augmented image saved successfully as E02_t0_NaiveCult_s31_channel3_aug_008.jpg
Augmented image saved successfully as E02_t0_NaiveCult_s35_channel3_aug_009.jpg
Augmented image saved successfully as E02_t0_NaiveCult_s37_channel3_aug_010.jpg
Augmented image saved successfully as E02_t0_NaiveCult_s42_channel3_aug_011.jpg
Augmented image saved successfully as E02_t0_NaiveCult_s44_channel3_

C:\Users\hetey\AppData\Local\Temp\ipykernel_12776\3152397554.py:47: UserWarning: ../data/train/jpg/B/E03_t0_NaiveCult_s36_channel3_aug_021.jpg is a low contrast image
  io.imsave(augmented_path, augmented_image)
C:\Users\hetey\AppData\Local\Temp\ipykernel_12776\327187003.py:37: UserWarning: ../data/train/jpg/B/E04_t0_NaiveCult_s37_channel3.jpg is a low contrast image
  io.imsave(os.path.join(output_folder, image_name), io.imread(original_path))


Augmented image saved successfully as E04_t0_NaiveCult_s30_channel3_aug_029.jpg
Augmented image saved successfully as E04_t0_NaiveCult_s35_channel3_aug_030.jpg
Augmented image saved successfully as E04_t0_NaiveCult_s37_channel3_aug_031.jpg
Augmented image saved successfully as E04_t0_NaiveCult_s41_channel3_aug_032.jpg
Augmented image saved successfully as E04_t0_NaiveCult_s44_channel3_aug_033.jpg
Augmented image saved successfully as E04_t0_NaiveCult_s46_channel3_aug_034.jpg
Augmented image saved successfully as E07_t7_NaiveCult_s04_channel3_aug_035.jpg
Augmented image saved successfully as E07_t7_NaiveCult_s17_channel3_aug_036.jpg
Augmented image saved successfully as E07_t7_NaiveCult_s18_channel3_aug_037.jpg
Augmented image saved successfully as E07_t7_NaiveCult_s23_channel3_aug_038.jpg
Augmented image saved successfully as E07_t7_NaiveCult_s29_channel3_aug_039.jpg
Augmented image saved successfully as E07_t7_NaiveCult_s39_channel3_aug_040.jpg
Augmented image saved successfully as E0